In [1]:
import pandas as pd 
import datetime
data = pd.read_csv('newData.csv')
user_id_list = data['user_id'].unique()
print(user_id_list, len(user_id_list))

[    0     2     3 ... 34483 34484 34490] 11728


In [2]:
new_data = {'user_id': user_id_list, 'car_ownership': 0}
df = pd.DataFrame(new_data)
df

,user_id,car_ownership
0,0,0
1,2,0
2,3,0
3,8,0
4,10,0
...,...,...
11723,34479,0
11724,34481,0
11725,34483,0
11726,34484,0


In [3]:
tripleg = pd.read_csv('triplegs.csv')
stop_points = pd.read_csv('staypoints.csv')

In [4]:
stop_points['started_at'] = pd.to_datetime(stop_points['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [5]:
ID_list=stop_points['user_id'].unique()
len(ID_list)

11728

In [6]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import trackintel as ti
import numpy as np
tpls = ti.io.file.read_triplegs_csv('trip_5.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)

In [7]:
# Predict the transport mode of triplegs.
"""simple-coarse method includes {'slow_mobility', 'motorized_mobility', 'fast_mobility'}. 
In the default classification, slow_mobility (<15 km/h) includes transport modes such as walking or cycling, 
motorized_mobility (<100 km/h) modes such as car or train, 
and fast_mobility (>100 km/h) modes such as high-speed rail or airplanes."""
categories = {
    5/3.6: 'slow_mobility',  # walking
    180/3.6: 'motorized_mobility',  # car
    np.inf: 'fast_mobility'  # high-speed rail or airplane
}
mode_tpls  = tpls.as_triplegs.predict_transport_mode(categories = categories)
mode_tpls

/usr/local/lib/python3.9/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


,user_id,tripleg_ID,started_at,finished_at,geom,mode
0,10,758,2022-10-07 22:00:05-04:00,2022-10-07 22:00:34-04:00,"LINESTRING (-81.65050 30.47010, -81.63820 30.4...",motorized_mobility
1,10,764,2022-10-08 13:45:17-04:00,2022-10-08 13:45:27-04:00,"LINESTRING (-81.63415 30.47833, -81.63416 30.4...",slow_mobility
2,10,770,2022-10-09 13:00:02-04:00,2022-10-09 13:00:51-04:00,"LINESTRING (-81.62109 30.47463, -81.62109 30.4...",slow_mobility
3,10,797,2022-10-11 22:00:32-04:00,2022-10-11 22:00:46-04:00,"LINESTRING (-81.65050 30.47010, -81.65050 30.4...",slow_mobility
4,14,930,2022-09-11 16:42:15-04:00,2022-09-11 16:42:40-04:00,"LINESTRING (-82.63589 29.44469, -82.63659 29.4...",motorized_mobility
...,...,...,...,...,...,...
40277,34471,837931,2022-10-25 20:03:34-04:00,2022-10-25 20:10:21-04:00,"LINESTRING (-82.80514 30.31518, -82.80514 30.3...",motorized_mobility
40278,34471,837932,2022-10-25 20:25:42-04:00,2022-10-25 20:27:53-04:00,"LINESTRING (-82.57936 29.97980, -82.56936 29.9...",motorized_mobility
40279,34471,837934,2022-10-30 01:52:42-04:00,2022-10-30 01:54:53-04:00,"LINESTRING (-82.78531 30.29825, -82.81831 30.3...",motorized_mobility
40280,34471,837935,2022-10-30 02:08:36-04:00,2022-10-30 02:08:46-04:00,"LINESTRING (-82.98503 30.48014, -82.98502 30.4...",slow_mobility


In [8]:
mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=mode_tpls, metric='count', per_user=True)
mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
10,0,1,3
14,0,1,0
22,2,73,29
30,0,1,0
32,0,0,1
...,...,...,...
34391,0,1,0
34405,0,5,0
34406,0,3,0


In [9]:
index = mode_split.index.values
for id in index:
    motor_count = mode_split.loc[id, 'fast_mobility'] + mode_split.loc[id, 'motorized_mobility'] + mode_split.loc[id, 'slow_mobility']
    if mode_split.loc[id, 'motorized_mobility'] / motor_count > 0.5:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,2,0
2,3,0
3,8,0
4,10,0
...,...,...
11723,34479,0
11724,34481,0
11725,34483,0
11726,34484,0


In [10]:
print(len(df[df['car_ownership'] == 1]))

2349


In [11]:
home_location = pd.read_csv('home_location.csv', index_col=0)
home_location

,ID,LAT,LON,LAT-4326,LON-4326
0,0,3550540.0,-9092080.0,30.364891,-81.675544
1,2,3458860.0,-9093020.0,29.651722,-81.683988
2,3,3456360.0,-9163720.0,29.632203,-82.319097
3,8,3510020.0,-9105740.0,30.050323,-81.798254
4,10,3565080.0,-9087560.0,30.477524,-81.634940
...,...,...,...,...,...
10869,34266,3473240.0,-9144240.0,29.763921,-82.144106
10870,34268,3464400.0,-9053100.0,29.694962,-81.325381
10871,34342,3526560.0,-9204640.0,30.178849,-82.686688
10872,34373,3541500.0,-9103240.0,30.294798,-81.775796


In [12]:
from geopy.distance import geodesic
from shapely.wkt import loads
from tqdm import tqdm
data_to_append_list = []
home_id_list = home_location['ID'].unique()
for id in tqdm(home_id_list, total = len(home_id_list)):
    # print(home_location.loc[home_location['ID'] == id, 'LAT-4326'].values[0])
    p1 = (home_location.loc[home_location['ID'] == id, "LAT-4326"].values[0], home_location.loc[home_location['ID'] == id, "LON-4326"].values[0])
    # print(p1)
    for sp in stop_points[stop_points['user_id'] == id].itertuples():
        p = loads(sp[5])
        lat = p.y
        lon = p.x
        p2 = (lat, lon)
        # print(p2)
        if geodesic(p1, p2).meters < 100:
            data_to_append_list.append(sp)
new_staypoints = pd.DataFrame.from_records(data_to_append_list)
new_staypoints


100%|██████████| 10874/10874 [01:37<00:00, 111.42it/s]


,0,1,2,3,4,5
0,2,2,2,2022-09-06 21:34:07-04:00,2022-09-06 22:09:29-04:00,POINT (-81.6840588550000177 29.6517638699999999)
1,5,5,2,2022-09-07 00:07:03-04:00,2022-09-07 00:38:53-04:00,POINT (-81.6840690000000080 29.6517566666666674)
2,8,8,2,2022-09-10 02:11:50-04:00,2022-09-10 03:00:43-04:00,POINT (-81.6840577100000189 29.6517777400000000)
3,10,10,2,2022-09-10 21:35:04-04:00,2022-09-10 22:09:37-04:00,POINT (-81.6840577100000189 29.6517348299999988)
4,12,12,2,2022-09-11 00:07:07-04:00,2022-09-11 00:38:57-04:00,POINT (-81.6840639275000058 29.6517739350000014)
...,...,...,...,...,...,...
98451,431671,431671,33744,2022-10-29 10:50:38-04:00,2022-10-29 10:59:48-04:00,POINT (-82.3449399999999940 29.5589300000000001)
98452,432136,432136,34054,2022-10-23 18:22:45-04:00,2022-10-23 18:39:11-04:00,POINT (-81.6337099999999936 30.2845100000000009)
98453,432405,432405,34266,2022-10-29 17:53:56-04:00,2022-10-29 18:21:27-04:00,POINT (-82.1440560000000062 29.7638719999999992)
98454,432501,432501,34342,2022-10-22 16:17:41-04:00,2022-10-22 16:45:48-04:00,POINT (-82.6863599999999934 30.1793300000000002)


In [13]:
drop_cols = [0,1]
new_staypoint = new_staypoints.drop(new_staypoints.columns[drop_cols], axis=1)
new_staypoint = new_staypoint.rename(columns={2:'user_id', 3: 'started_at', 4: 'finished_at', 5: 'geom'})
new_staypoint


,user_id,started_at,finished_at,geom
0,2,2022-09-06 21:34:07-04:00,2022-09-06 22:09:29-04:00,POINT (-81.6840588550000177 29.6517638699999999)
1,2,2022-09-07 00:07:03-04:00,2022-09-07 00:38:53-04:00,POINT (-81.6840690000000080 29.6517566666666674)
2,2,2022-09-10 02:11:50-04:00,2022-09-10 03:00:43-04:00,POINT (-81.6840577100000189 29.6517777400000000)
3,2,2022-09-10 21:35:04-04:00,2022-09-10 22:09:37-04:00,POINT (-81.6840577100000189 29.6517348299999988)
4,2,2022-09-11 00:07:07-04:00,2022-09-11 00:38:57-04:00,POINT (-81.6840639275000058 29.6517739350000014)
...,...,...,...,...
98451,33744,2022-10-29 10:50:38-04:00,2022-10-29 10:59:48-04:00,POINT (-82.3449399999999940 29.5589300000000001)
98452,34054,2022-10-23 18:22:45-04:00,2022-10-23 18:39:11-04:00,POINT (-81.6337099999999936 30.2845100000000009)
98453,34266,2022-10-29 17:53:56-04:00,2022-10-29 18:21:27-04:00,POINT (-82.1440560000000062 29.7638719999999992)
98454,34342,2022-10-22 16:17:41-04:00,2022-10-22 16:45:48-04:00,POINT (-82.6863599999999934 30.1793300000000002)


In [14]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_stop_by_user = new_staypoint[new_staypoint['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_related_row in related_stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_related_row[2])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

new_trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip'])
new_trip_df

,user_id,trip_started_at,trip_finished_at,trip
0,8,2022-09-10 00:59:14-04:00,2022-09-10 01:03:16-04:00,LINESTRING (-81.8096299999999985 30.0851299999...
1,8,2022-09-27 10:38:11-04:00,2022-09-27 10:38:22-04:00,LINESTRING (-81.8045299999999997 30.1029400000...
2,8,2022-09-30 20:47:50-04:00,2022-09-30 20:50:04-04:00,LINESTRING (-81.7980840000000029 30.0505470000...
3,22,2022-09-06 13:42:11-04:00,2022-09-06 13:48:23-04:00,LINESTRING (-81.4465212799999989 30.3217291799...
4,22,2022-09-07 13:13:10-04:00,2022-09-07 13:17:38-04:00,LINESTRING (-81.4755399999999952 30.3195699999...
...,...,...,...,...
10067,33747,2022-10-30 23:37:27-04:00,2022-10-30 23:39:03-04:00,LINESTRING (-81.7521500000000003 30.3784999999...
10068,33756,2022-10-31 14:25:55-04:00,2022-10-31 14:27:43-04:00,LINESTRING (-81.7386889499999967 30.2478718800...
10069,34160,2022-10-30 07:10:09-04:00,2022-10-30 07:10:59-04:00,LINESTRING (-83.1324079999999981 29.6222519999...
10070,34160,2022-10-30 07:19:14-04:00,2022-10-30 07:19:23-04:00,LINESTRING (-83.1324079999999981 29.6222519999...


In [15]:
new_trip_df.to_csv('home_related_trip.csv')

In [16]:
new_tpls = ti.io.file.read_triplegs_csv('home_related_trip.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)
new_mode_tpls  = new_tpls.as_triplegs.predict_transport_mode(categories = categories)
new_mode_tpls

,user_id,started_at,finished_at,geom,mode
0,8,2022-09-10 00:59:14-04:00,2022-09-10 01:03:16-04:00,"LINESTRING (-81.80963 30.08513, -81.79732 30.1...",motorized_mobility
1,8,2022-09-27 10:38:11-04:00,2022-09-27 10:38:22-04:00,"LINESTRING (-81.80453 30.10294, -81.80454 30.1...",slow_mobility
2,8,2022-09-30 20:47:50-04:00,2022-09-30 20:50:04-04:00,"LINESTRING (-81.79808 30.05055, -81.79843 30.0...",slow_mobility
3,22,2022-09-06 13:42:11-04:00,2022-09-06 13:48:23-04:00,"LINESTRING (-81.44652 30.32173, -81.44652 30.3...",motorized_mobility
4,22,2022-09-07 13:13:10-04:00,2022-09-07 13:17:38-04:00,"LINESTRING (-81.47554 30.31957, -81.49532 30.3...",motorized_mobility
...,...,...,...,...,...
10067,33747,2022-10-30 23:37:27-04:00,2022-10-30 23:39:03-04:00,"LINESTRING (-81.75215 30.37850, -81.75221 30.3...",slow_mobility
10068,33756,2022-10-31 14:25:55-04:00,2022-10-31 14:27:43-04:00,"LINESTRING (-81.73869 30.24787, -81.73533 30.2...",motorized_mobility
10069,34160,2022-10-30 07:10:09-04:00,2022-10-30 07:10:59-04:00,"LINESTRING (-83.13241 29.62225, -83.13273 29.6...",motorized_mobility
10070,34160,2022-10-30 07:19:14-04:00,2022-10-30 07:19:23-04:00,"LINESTRING (-83.13241 29.62225, -83.13243 29.6...",slow_mobility


In [17]:
new_mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=new_mode_tpls, metric='count', per_user=True)
new_mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
8,0,1,2
22,1,23,20
30,0,0,1
74,0,6,2
109,0,6,1
...,...,...,...
33744,0,1,0
33747,0,8,11
33756,0,1,0


In [18]:
index = new_mode_split.index.values
for id in index:
    motor_count = new_mode_split.loc[id, 'motorized_mobility']
    if motor_count > 0:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,2,0
2,3,0
3,8,1
4,10,0
...,...,...
11723,34479,0
11724,34481,0
11725,34483,0
11726,34484,0


In [19]:
print(len(df[df['car_ownership'] == 1]))

2819


In [20]:
tpls = ti.io.file.read_triplegs_csv('triplegs.csv', index_col=0)
all_tpls = tpls[tpls['user_id'].isin(user_id_list)]

In [21]:
len(all_tpls['user_id'].unique())

11057

In [22]:
all_mode_tpls  = all_tpls.as_triplegs.predict_transport_mode(categories=categories)
all_mode_tpls

/usr/local/lib/python3.9/site-packages/trackintel/geogr/distances.py:227: UserWarning: The CRS of your data is not defined.
  warnings.warn("The CRS of your data is not defined.")


,user_id,started_at,finished_at,geom,mode
id,,,,,
6,2,2022-09-06 22:19:08-04:00,2022-09-06 22:19:58-04:00,"LINESTRING (-81.68406 29.65175, -81.68407 29.6...",slow_mobility
7,2,2022-09-06 22:31:37-04:00,2022-09-06 22:37:57-04:00,"LINESTRING (-81.68406 29.65175, -81.68407 29.6...",motorized_mobility
8,2,2022-09-07 01:57:53-04:00,2022-09-07 01:58:57-04:00,"LINESTRING (-81.65829 29.65349, -81.65827 29.6...",slow_mobility
9,2,2022-09-07 04:46:05-04:00,2022-09-07 04:46:50-04:00,"LINESTRING (-81.68406 29.65178, -81.68405 29.6...",slow_mobility
10,2,2022-09-07 10:25:44-04:00,2022-09-07 10:29:51-04:00,"LINESTRING (-81.65829 29.65349, -81.68406 29.6...",motorized_mobility
...,...,...,...,...,...
547547,34471,2022-10-30 02:06:15-04:00,2022-10-30 02:12:24-04:00,"LINESTRING (-82.95013 30.45929, -82.95011 30.4...",motorized_mobility
547548,34471,2022-10-30 02:50:13-04:00,2022-10-30 02:50:24-04:00,"LINESTRING (-83.15011 30.61566, -83.15011 30.6...",slow_mobility
547549,34472,2022-10-26 23:03:49-04:00,2022-10-26 23:04:01-04:00,"LINESTRING (-81.54434 30.32230, -81.55156 30.3...",fast_mobility


In [23]:
all_mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=all_mode_tpls, metric='count', per_user=True)
all_mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
2,0,11,16
3,0,6,91
8,0,8,144
10,0,16,152
11,0,1,22
...,...,...,...
34468,0,0,5
34471,0,12,2
34472,1,0,0


In [24]:
index = all_mode_split.index.values
for id in index:
    motor_count = all_mode_split.loc[id, 'motorized_mobility']
    if motor_count > 0:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,2,1
2,3,1
3,8,1
4,10,1
...,...,...
11723,34479,0
11724,34481,0
11725,34483,0
11726,34484,0


In [25]:
print(len(df[df['car_ownership'] == 1]))

7177


In [26]:
filter_mode = all_mode_split[(all_mode_split['fast_mobility'] + all_mode_split['motorized_mobility'] + all_mode_split['slow_mobility']) > 20 ]
filter_mode

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
2,0,11,16
3,0,6,91
8,0,8,144
10,0,16,152
11,0,1,22
...,...,...,...
34074,0,2,21
34187,0,2,27
34264,0,0,43


In [27]:
filter_mode.shape[0]

5359

In [28]:
test = filter_mode[filter_mode['motorized_mobility'] > 0]

In [29]:
test

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
2,0,11,16
3,0,6,91
8,0,8,144
10,0,16,152
11,0,1,22
...,...,...,...
33848,0,2,27
33947,0,3,23
34074,0,2,21


In [30]:
percentage = len(filter_mode[(filter_mode['motorized_mobility']) > 0]) / filter_mode.shape[0]
percentage_string = "{:.0%}".format(percentage)
print(percentage_string)

75%


In [31]:
home_locations = home_location.rename(columns={'ID':'user_id'})
home_locations
merged_data = pd.merge(df, home_locations[['user_id','LAT-4326','LON-4326']], on='user_id', how='left')
merged_data

,user_id,car_ownership,LAT-4326,LON-4326
0,0,0,30.364891,-81.675544
1,2,1,29.651722,-81.683988
2,3,1,29.632203,-82.319097
3,8,1,30.050323,-81.798254
4,10,1,30.477524,-81.634940
...,...,...,...,...
11723,34479,0,29.767976,-82.506127
11724,34481,0,30.212080,-81.772562
11725,34483,0,30.000546,-81.370836
11726,34484,0,30.368612,-81.668897


In [32]:
merged_data.to_csv('User.csv')